In [ ]:
model_name = "/kaggle/input/llama2-qlora"

In [ ]:
import sys
sys.path.append("/kaggle/input/sentence-transformers-222/sentence-transformers")
!pip -q install /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip -q install /kaggle/input/transformers431/transformers-4.31.0-py3-none-any.whl
!pip -q install /kaggle/input/bitsandbytes-0410/bitsandbytes-0.41.0-py3-none-any.whl
!pip -q install /kaggle/input/llm-lib/peft-0.5.0-py3-none-any.whl
!pip -q install /kaggle/input/llm-lib/accelerate-0.21.0-py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
import ctypes
import pandas as pd
import numpy as np
import glob
import faiss  # Facebook AI Similarity Search，Facebook开源的一个高效相似度搜索库
from joblib import Parallel, delayed
import heapq
import pickle
import gc  # Python的垃圾回收模块
import sys
from tqdm import tqdm  # 用于显示进度条
from faiss import write_index, read_index

sys.path.append("/kaggle/input/sentence-transformers-222/sentence-transformers")
libc = ctypes.CDLL("libc.so.6")  # 加载C库

def get_context():
    # 读取sub_df
    sub_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
    if len(sub_df) == 200:  # 如果sub_df的长度为200，则将sub_df替换为train.csv中的数据
        sub_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
        print("read train.csv")
    # 将prompt, A, B, C, D, E字段的内容连接起来，作为all_text字段的内容
    sub_df['all_text'] = sub_df.apply(lambda x: " ".join([x['prompt'], x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)
    print(f"sub_df.shape: {sub_df.shape}")

    # 创建 embed 模型
    from sentence_transformers import SentenceTransformer
    SIM_MODEL = f'/kaggle/input/bge-large-en-v1-5/bge-large-en-v1.5'
    model = SentenceTransformer(SIM_MODEL, device='cuda')  # 使用SentenceTransformer模型，设备为cuda
    model = model.half()  # 将模型的参数转为半精度浮点数（float16）

    # 对sub_df进行embedding
    embeds = []
    for all_text in sub_df['all_text'].tolist():
        embeds.append(model.encode(all_text, show_progress_bar=False))  # 对all_text字段的内容进行embedding
    embeds = np.array(embeds)
    print(f"{embeds.shape=}")
    sub_df["embeds"] = embeds.tolist()  # 将embedding的结果添加到sub_df的embeds字段中

    sub_emb = np.stack(sub_df["embeds"]).astype(np.float32)
    print(f"sub_emb.shape: {sub_emb.shape}")

    # 文件来自于 data_preprocess.ipynb
    context_path = f"/kaggle/input/wiki-270k/wiki-270k-sentences.parquet"
    context_index_path = f"/kaggle/input/wiki-270k/wiki-270k.index"

    # 为每个题目加上寻找最相似的n个context段落
    NUM_ARTICLES = 5
    context_index = read_index(context_index_path)  # 加载wiki270k的index
    print(f"{context_index.ntotal=}")

    print("Searching...")
    # 使用faiss进行最近邻搜索，找到与每个题目最相似的5篇wiki文章
    score, all_test_wiki_indices = context_index.search(sub_emb, NUM_ARTICLES) 

    context_df = pd.read_parquet(context_path, columns=["title", "all_text"])  # 读取wiki文章的标题和内容
    all_test_texts = []

    for wiki_indices in all_test_wiki_indices:
        texts = context_df.iloc[wiki_indices].all_text.values  # 获取最相似的文章的内容
        all_test_texts.append(texts.tolist())

    sub_df["context"] = all_test_texts  # 将最相似的文章的内容添加到sub_df的context字段中
    gc.collect()  # 执行垃圾回收

    # 保存 sub_df 文件
    sub_df.to_parquet(f"/kaggle/working/all_data.parquet", engine='pyarrow')


get_context()

In [ ]:
import psutil

# 获取虚拟内存的统计信息
memory_info = psutil.virtual_memory()

# 打印内存占用情况
print(f"总内存：{memory_info.total / (1024**2):.2f} MB")
print(f"已使用内存：{memory_info.used / (1024**2):.2f} MB")
print(f"空闲内存：{memory_info.available / (1024**2):.2f} MB")
print(f"内存使用百分比：{memory_info.percent}%")

# 大模型

In [ ]:
import sys
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from transformers import LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer
import transformers
from transformers import BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM

transformers.__version__

In [ ]:
# 导入AutoTokenizer，这是HuggingFace库中的一个类，它可以自动地从预训练模型中加载对应的tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,  # 预训练模型的名称
    use_fast=True,  # 使用快速tokenizer
    trust_remote_code=True,  # 信任远程代码，如果模型包含自定义的tokenizer代码，则执行该代码
    truncation_side="left",  # 如果文本超过模型的最大长度，那么从文本的左边开始截断
)

# 定义一个函数，该函数用于格式化数据集中的一行数据
def format_instruction(row, mode):
    # 初始化一个空字符串
    text = ""
    # 在字符串中添加一些介绍性的文本
    text += "The following are multiple choice questions (with answers) that include context\n"
    text += "\n"
    text += "Context:\n"
    # 从数据集的一行数据中提取出前五个上下文，并添加到字符串中
    for i in range(5):
        text += f"{row['context'][i]}\n"
        if i != 4:
            text += "###\n"
    text += "\n"
    text += "Question:\n"
    # 从数据集的一行数据中提取出问题，并添加到字符串中
    text += f"{row['prompt']}\n"
    text += "\n"
    text += "Options:\n"
    # 从数据集的一行数据中提取出选项，并添加到字符串中
    text += f"A: {row['A']}\n"
    text += f"B: {row['B']}\n"
    text += f"C: {row['C']}\n"
    text += f"D: {row['D']}\n"
    text += f"E: {row['E']}\n"
    text += "\n"
    text += "Answer: "

    # 如果是训练模式，那么还需要从数据集的一行数据中提取出答案，并添加到字符串中
    if mode == "train":
        text += f"{row['answer']}"
    return text  # 返回格式化后的字符串

# 使用pandas库中的read_parquet函数，从.parquet文件中读取数据，返回一个DataFrame对象
df = pd.read_parquet(f"/kaggle/working/all_data.parquet")
# 对DataFrame的每一行应用format_instruction函数，将返回的字符串添加到新的一列中
df['instruction'] = df.apply(lambda row: format_instruction(row, "test"), axis=1)

# 打印DataFrame的第一行的instruction列的值
print(df.iloc[0].instruction)


## 模型

In [ ]:
# 使用预训练模型创建一个自动Peft模型，用于因果语言模型（Causal Language Modeling）
# 因果语言模型是一种自然语言处理模型，它可以预测给定前文的情况下下一个词是什么
model = AutoPeftModelForCausalLM.from_pretrained(
    model_name,  # 预训练模型的名称，这个名称通常是预训练模型在Hugging Face Model Hub上的标识符

    # "device_map"参数用于指定模型运行的设备。"auto"表示自动选择最佳设备，如果有GPU，则优先选择GPU，否则选择CPU
    device_map="auto",  
   
    # "low_cpu_mem_usage"参数设置为True，表示在CPU内存使用方面进行优化，尽可能减少CPU内存的使用
    low_cpu_mem_usage=True,
    
    # "torch_dtype"参数指定了模型中张量的数据类型。在这里，我们使用的是半精度浮点数（float16），
    torch_dtype=torch.float16,
    
    # "load_in_4bit"参数如果设置为True，模型将以4位的精度加载，这可以进一步减少内存使用和计算时间，但可能会降低模型的精度
    # load_in_4bit=True,
    )

model = model.eval()

# 将tokenizer的pad_token（填充符）设置为eos_token（句子结束符）。
tokenizer.pad_token = tokenizer.eos_token



In [ ]:
import psutil

# 获取虚拟内存的统计信息
memory_info = psutil.virtual_memory()

# 打印内存占用情况
print(f"总内存：{memory_info.total / (1024**2):.2f} MB")
print(f"已使用内存：{memory_info.used / (1024**2):.2f} MB")
print(f"空闲内存：{memory_info.available / (1024**2):.2f} MB")
print(f"内存使用百分比：{memory_info.percent}%")

## 推理

In [ ]:
preds = []  # 初始化一个空列表，用于存储预测结果

# 对数据框 df 的每一行进行迭代处理
for _, row in tqdm(df.iterrows(), total=len(df)):
    # 使用 tokenizer 处理每一行中的 'instruction' 列的文本
    # return_tensors="pt" 表示返回的是 PyTorch 张量
    # truncation=True 表示如果文本超过模型的最大长度，就截断它
    # max_length=2048 是设置的最大长度，如果文本的长度超过这个值，就会被截断
    inputs = tokenizer(
        row['instruction'], 
        return_tensors="pt",
        truncation=True, 
        max_length=2048, 
    ).to(f"cuda:{model.device.index}")

    # 不计算梯度，以节省计算资源，因为这里只是进行推理，不需要更新模型的参数
    with torch.no_grad():
        # 使用模型处理输入，并获取输出的 logits
        # logits 是模型最后一层的输出，通常用于计算概率分布
        output = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"]
            ).logits

    # 找出 attention_mask 中非零元素的索引，因为 attention_mask 中的非零元素对应的是输入的有效 token
    non_zero_indices = torch.nonzero(inputs['attention_mask'])
    # 获取最后一个有效 token 的索引
    last_one_index = non_zero_indices[-1][1].item()
    # 获取最后一个有效 token 对应的 logits
    first_token_probs = output[0][last_one_index]

    # 对每一个选项（' A'，' B'，' C'，' D'，' E'）进行处理
    # tokenizer(option).input_ids[-1] 是获取选项对应的 id
    # first_token_probs[id] 是获取该 id 对应的 logits
    # 然后将 logits 和选项一起作为一个元组存储在 options_list 中
    options_list = [
        (first_token_probs[tokenizer(' A').input_ids[-1]], 'A'),
        (first_token_probs[tokenizer(' B').input_ids[-1]], 'B'),
        (first_token_probs[tokenizer(' C').input_ids[-1]], 'C'),
        (first_token_probs[tokenizer(' D').input_ids[-1]], 'D'),
        (first_token_probs[tokenizer(' E').input_ids[-1]], 'E'),
    ]
    # 对 options_list 进行排序，排序的依据是 logits，也就是每个选项的概率
    options_list = sorted(options_list, reverse=True)
    
    # 初始化一个空列表，用于存储前三个最可能的选项
    pred = []
    for i in range(3):
        # 添加最可能的选项到 pred 中
        pred.append(options_list[i][1])
    # 将 pred 中的选项用空格连接成一个字符串
    pred = ' '.join(pred)
    # 将预测结果添加到 preds 中
    preds.append(pred)
    
# 将 preds 添加到 df 的 'prediction' 列中
df['prediction'] = preds

# 将 df 的 'id' 列和 'prediction' 列保存到 'submission.csv' 文件中
df[["id", "prediction"]].to_csv('submission.csv', index=False)